# 1. 라이브러리 버전 확인

In [1]:
import tensorflow
import numpy
import transformers
import datasets

print(tensorflow.__version__)
print(numpy.__version__)
print(transformers.__version__)
print(datasets.__version__)

2.6.0
1.21.4
4.11.3
1.14.0


# STEP 1. NSMC 데이터 분석 및 Huggingface dataset 구성
---
데이터셋은 깃허브에서 다운받거나, Huggingface datasets에서 가져올 수 있습니다. 앞에서 배운 방법들을 활용해봅시다!

In [3]:
# import datasets
from datasets import load_dataset
from datasets import list_datasets, load_dataset, list_metrics, load_metric
from huggingface_hub.hf_api import HfFolder


huggingface_nsmc_dataset = load_dataset('nsmc')
print(huggingface_nsmc_dataset)

Downloading:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/807 [00:00<?, ?B/s]

Using custom data configuration default


  0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/6.33M [00:00<?, ?B/s]

...

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset nsmc downloaded and prepared to /aiffel/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 50000
    })
})


In [4]:
train = huggingface_nsmc_dataset['train']
cols = train.column_names
cols

['id', 'document', 'label']

In [5]:
train

Dataset({
    features: ['id', 'document', 'label'],
    num_rows: 150000
})

In [6]:
test = huggingface_nsmc_dataset['test']
test

Dataset({
    features: ['id', 'document', 'label'],
    num_rows: 50000
})

In [7]:
for i in range(5):
    for col in cols:
        print(col, ":", train[col][i])
    print('\n')

id : 9976970
document : 아 더빙.. 진짜 짜증나네요 목소리
label : 0


id : 3819312
document : 흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
label : 1


id : 10265843
document : 너무재밓었다그래서보는것을추천한다
label : 0


id : 9045019
document : 교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
label : 0


id : 6483659
document : 사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다
label : 1




In [8]:
len(train)

150000

In [9]:
for i in range(5):
    for col in cols:
        print(col, ":", test[col][i])
    print('\n')

id : 6270596
document : 굳 ㅋ
label : 1


id : 9274899
document : GDNTOPCLASSINTHECLUB
label : 0


id : 8544678
document : 뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아
label : 0


id : 6825595
document : 지루하지는 않은데 완전 막장임... 돈주고 보기에는....
label : 0


id : 6723715
document : 3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??
label : 0




- train/test 동일한 형식으로 되어있음
- train 150k test 50k로 되어있음

# STEP 2. klue/bert-base model 및 tokenizer 불러오기
---


In [10]:
import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_checkpoint = "klue/bert-base"
num_labels = 2
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)


Downloading:   0%|          | 0.00/289 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/425 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/424M [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized

# STEP 3. 위에서 불러온 tokenizer으로 데이터셋을 전처리하고, model 학습 진행해 보기
---



In [13]:
import pandas as pd
len(train['document'])

train_dataset = pd.DataFrame({'id':train['id'],
                             'document':train['document'],
                             'label':train['label']})

test_dataset_none = pd.DataFrame({'id':test['id'],
                             'document':test['document'],
                             'label':test['label']})

# student_card
train_dataset[:5]
val_dataset = test_dataset_none[:int(len(test_dataset_none)/2)]
test_dataset = test_dataset_none[int(len(test_dataset_none)/2):]

In [14]:
print(len(train_dataset))
print(len(val_dataset))
print(len(test_dataset))

150000
25000
25000


In [15]:
train_dataset

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [16]:
from datasets import Dataset

# DataFrame 데이터를 dict 내부에 list로 변경
train_dataset = train_dataset.to_dict(orient='list')
val_dataset = val_dataset.to_dict('list')
test_dataset = test_dataset.to_dict('list')

# Hugging Face dataset으로 변환
tf_train_dataset = Dataset.from_dict(train_dataset)
tf_val_dataset = Dataset.from_dict(val_dataset)
tf_test_dataset = Dataset.from_dict(test_dataset)

In [19]:
from datasets import Dataset, DatasetDict

# 분할된 데이터셋을 DatasetDict 형태로 조합
hf_dataset_dict = DatasetDict({
    'train': tf_train_dataset,
    'validation': tf_val_dataset,
    'test': tf_test_dataset
})

In [20]:
hf_dataset_dict

DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 150000
    })
    validation: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 25000
    })
})

In [21]:
# 훈련 데이터셋 확인
for i in range(5):
    for column in hf_dataset_dict['train'].column_names:
        print(column, ":", hf_dataset_dict['train'][column][i])
    print('\n')

id : 9976970
document : 아 더빙.. 진짜 짜증나네요 목소리
label : 0


id : 3819312
document : 흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
label : 1


id : 10265843
document : 너무재밓었다그래서보는것을추천한다
label : 0


id : 9045019
document : 교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
label : 0


id : 6483659
document : 사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다
label : 1




In [22]:
# 훈련데이터로부터 샘플을 불러와 토크나이저 적용

# 변환 함수 정의
def transform(data):
    return tokenizer(
        data['document'],
        truncation=True,
        padding='max_length',
        max_length = 80,
        return_token_type_ids=False,
    )


In [23]:
len(huggingface_nsmc_dataset)

2

In [24]:
hf_dataset = huggingface_nsmc_dataset.map(transform, batched=True)

  0%|          | 0/150 [00:00<?, ?ba/s]

  0%|          | 0/50 [00:00<?, ?ba/s]

In [25]:
hf_dataset = hf_dataset_dict.map(transform, batched=True)

  0%|          | 0/150 [00:00<?, ?ba/s]

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/25 [00:00<?, ?ba/s]

In [27]:
# hf_train_dataset
train
test

Dataset({
    features: ['id', 'document', 'label'],
    num_rows: 50000
})

In [28]:
# 훈련, 검증, 테스트 데이터셋 분할
hf_train_dataset = hf_dataset['train']
hf_val_dataset = hf_dataset['validation']
hf_test_dataset = hf_dataset['test']

# hf_train_dataset = hf_dataset['train']

# hf_test_dataset = hf_dataset['test']

In [29]:
import os
import numpy as np
from transformers import Trainer, TrainingArguments

output_dir = os.getenv('HOME')+'/aiffel/prj'

training_arguments = TrainingArguments(
    output_dir,                                         # output이 저장될 경로
    evaluation_strategy="epoch",           #evaluation하는 빈도
    learning_rate = 2e-5,                         #learning_rate
    per_device_train_batch_size = 16,   # 각 device 당 batch size
    per_device_eval_batch_size = 16,    # evaluation 시에 batch size
    num_train_epochs = 3,                     # train 시킬 총 epochs
    weight_decay = 0.01,                        # weight decay
)

In [30]:
from datasets import load_metric
metric = load_metric('glue', 'mrpc')

def compute_metrics(eval_pred):    
    predictions,labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references = labels)

Downloading:   0%|          | 0.00/1.86k [00:00<?, ?B/s]

In [ ]:
trainer = Trainer(
    model=model,           # 학습시킬 model
    args=training_arguments,           # TrainingArguments을 통해 설정한 arguments
    train_dataset=hf_train_dataset,    # training dataset
    eval_dataset=hf_val_dataset,       # evaluation dataset
    compute_metrics=compute_metrics,
)
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: id, document.
***** Running training *****
  Num examples = 150000
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 28125


Epoch,Training Loss,Validation Loss


Saving model checkpoint to /aiffel/aiffel/prj/checkpoint-500
Configuration saved in /aiffel/aiffel/prj/checkpoint-500/config.json
Model weights saved in /aiffel/aiffel/prj/checkpoint-500/pytorch_model.bin


In [ ]:
# 테스트 데이터로 평가
trainer.evaluate(hf_test_dataset)

# STEP 4. Fine-tuning을 통하여 모델 성능(accuarcy) 향상시키기
---
데이터 전처리, TrainingArguments 등을 조정하여 모델의 정확도를 90% 이상으로 끌어올려봅시다.

# STEP 5. Bucketing을 적용하여 학습시키고, STEP 4의 결과와의 비교
---
- 아래 링크를 바탕으로 bucketing과 dynamic padding이 무엇인지 알아보고, 이들을 적용하여 model을 학습시킵니다.

    - Data Collator

    - Trainer.TrainingArguments 의 group_by_length

- STEP 4에 학습한 결과와 bucketing을 적용하여 학습시킨 결과를 비교해보고, 모델 성능 향상과 훈련 시간 두 가지 측면에서 각각 어떤 이점이 있는지 비교해봅시다.